In [ ]:
import math
from smt.sampling_methods import LHS
import torch              
import torch.nn as nn                                   
from collections import OrderedDict
import numpy as np


In [37]:
# CUDA GPU selection
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

cuda


In [46]:
# Definition of Deep Neural Network Architecture
class DNNModel(torch.nn.Module):
    def __init__(
        self,
        input_size,
        hidden_size,
        output_size,
        depth,
        act=torch.nn.Tanh,
        init_mode='xavier'
    ):
        super(DNNModel, self).__init__()
        
        layers = [('input', torch.nn.Linear(input_size, hidden_size))]
        layers.append(('input_activation', act()))
        for i in range(depth): 
            layers.append(
                ('hidden_%d' % i, torch.nn.Linear(hidden_size, hidden_size))
            )
            layers.append(('activation_%d' % i, act()))
        layers.append(('output', torch.nn.Linear(hidden_size, output_size)))

        layerDict = OrderedDict(layers)
        self.layers = torch.nn.Sequential(layerDict)

        if init_mode == 'xavier':
          for m in self.modules():                
              if isinstance(m, nn.Linear):
                  size = m.weight.size()
                  fan_out = size[0] # Number of rows
                  fan_in = size[1] # Number of columns
                  variance = math.sqrt(2.0/(fan_in+fan_out))
                  m.weight.data.normal_(0.0, variance)
                  if m.bias is not None:
                      m.bias.data.normal_(0, variance)            

    def forward(self, x):
        out = self.layers(x)
        return out

In [47]:
# Defintion of Physics infomed neural network
class PINN():
    def __init__(self):
        
        self.model = DNNModel(
            input_size=2,
            hidden_size=50,
            output_size=5,
            depth=8,
            act=torch.nn.Tanh,
            init_mode='xavier'
            ).to(device)
        
        self.dx = 0.01
        self.dy = 0.005
        self.nx = 500
        self.ny = 200
        self.rho=1.1644        
        self.mu=0.005
        
        x = torch.arange(0, 5 + self.dx, self.dx)
        y = torch.arange(0, 1 + self.dy, self.dy)

        xlimits = np.array([[0.001, 0.0], [4.999, 0.0]])
        sampling = LHS(xlimits=xlimits)        
        xb = sampling(self.nx)

        xb=torch.tensor(xb,dtype=torch.float32)        

        xlimits = np.array([[0.0, 0.001], [0.0, 0.999]])
        sampling = LHS(xlimits=xlimits)        
        yb = sampling(self.ny)

        yb=torch.tensor(yb,dtype=torch.float32)        
        
        # Geometry and Discretization training data
        self.inlet=torch.stack(torch.meshgrid(torch.zeros(1), y)).reshape(2, -1).T
        self.outlet=torch.stack(torch.meshgrid(torch.ones(1)*5, y)).reshape(2, -1).T

        bc1 = torch.stack(torch.meshgrid(x, torch.zeros(1))).reshape(2, -1).T
        bc2 = torch.stack(torch.meshgrid(x, torch.ones(1))).reshape(2, -1).T

        self.id = torch.stack(torch.meshgrid(xb[:,1], yb[:,1])).reshape(2, -1).T
        self.bc = torch.cat([bc1, bc2])

        u_inlet = torch.ones(len(self.inlet))
        v_inlet = torch.zeros(len(self.inlet))
        
        v_outlet = torch.zeros(len(self.outlet))
        p_outlet = torch.zeros(len(self.outlet))
        
        u_bc1 = torch.zeros(len(bc1))
        v_bc1 = torch.zeros(len(bc1))

        u_bc2 = torch.zeros(len(bc2))
        v_bc2 = torch.zeros(len(bc2))           

        self.uv_inlet = torch.stack((u_inlet, v_inlet)).reshape(2, -1).T
        self.vp_outlet = torch.stack((v_outlet, p_outlet)).reshape(2, -1).T

        uv_bc1 = torch.stack((u_bc1, v_bc1)).reshape(2, -1).T
        uv_bc2 = torch.stack((u_bc2, v_bc2)).reshape(2, -1).T

        self.uv_bc = torch.cat([uv_bc1, uv_bc2])

        self.pde_sol=torch.zeros(len(self.id))

        self.y_train = torch.cat([self.uv_inlet, uv_bc1, uv_bc2, self.vp_outlet])
        self.y_train = self.y_train.unsqueeze(1)
        
        self.id = self.id.to(device)
        self.bc = self.bc.to(device)
        self.inlet = self.inlet.to(device)
        self.outlet = self.outlet.to(device)
        self.uv_inlet = self.uv_inlet.to(device)
        self.vp_outlet = self.vp_outlet.to(device)
        self.pde_sol = self.pde_sol.to(device)
        self.uv_bc = self.uv_bc.to(device)
        self.y_train = self.y_train.to(device)
        self.id.requires_grad = True
        self.bc.requires_grad = True
        self.inlet.requires_grad = True
        self.outlet.requires_grad = True
        
        self.criterion = torch.nn.MSELoss()
        self.iter = 1
        
        self.optimizer = torch.optim.LBFGS(
            self.model.parameters(), 
            lr=0.001, 
            max_iter=100000, 
            max_eval=100000, 
            history_size=50,
            tolerance_grad=1e-5, 
            tolerance_change=1.0 * np.finfo(float).eps
            )
        
        self.adam = torch.optim.Adam(self.model.parameters(), lr=0.0001)

    #Definition of Bounday and Navier-stokes residual loss   
    def loss_func(self):
        self.optimizer.zero_grad()
        
        y_inlet = self.model(self.inlet)        
        dpsi_dX = torch.autograd.grad(inputs=self.inlet, outputs=y_inlet[:,0], grad_outputs=torch.ones_like(y_inlet[:,0]), retain_graph=True, create_graph=True)[0]
        dpsi_dx = -1*dpsi_dX[:, 0] 
        dpsi_dy = dpsi_dX[:, 1]       
        loss_inlet = self.criterion(dpsi_dy, self.uv_inlet[:,0])+self.criterion(dpsi_dx, self.uv_inlet[:,1])
            
        y_bc = self.model(self.bc)
        dpsi_dX = torch.autograd.grad(inputs=self.bc, outputs=y_bc[:,0], grad_outputs=torch.ones_like(y_bc[:,0]), retain_graph=True, create_graph=True)[0]
        dpsi_dx = -1*dpsi_dX[:, 0] 
        dpsi_dy = dpsi_dX[:, 1]
        loss_bc = self.criterion(dpsi_dy, self.uv_bc[:,0])+self.criterion(dpsi_dx, self.uv_bc[:,1])

        y_outlet = self.model(self.outlet)
        loss_outlet = self.criterion(y_outlet[:,1], self.vp_outlet[:,1])
        
        loss_b=loss_inlet+loss_bc+loss_outlet

        y_id = self.model(self.id)
        dpsi_dX = torch.autograd.grad(inputs=self.id, outputs=y_id[:,0], grad_outputs=torch.ones_like(y_id[:,0]), retain_graph=True, create_graph=True)[0]
        dpsi_dx = -1*dpsi_dX[:, 0] 
        dpsi_dy = dpsi_dX[:, 1]
        dpsi_dXX = torch.autograd.grad(inputs=self.id, outputs=dpsi_dy, grad_outputs=torch.ones_like(dpsi_dy), retain_graph=True, create_graph=True)[0]
        dpsi_dyx = dpsi_dXX[:, 0] 
        dpsi_dyy = dpsi_dXX[:, 1]
        dpsi_dYY = torch.autograd.grad(inputs=self.id, outputs=dpsi_dx, grad_outputs=torch.ones_like(dpsi_dx), retain_graph=True, create_graph=True)[0]
        dpsi_dxx = dpsi_dYY[:, 0] 
        dpsi_dxy = dpsi_dYY[:, 1]
        dsxx_dX = torch.autograd.grad(inputs=self.id, outputs=y_id[:,2], grad_outputs=torch.ones_like(y_id[:,2]), retain_graph=True, create_graph=True)[0]
        dsxx_dx = dsxx_dX[:,0]
        dsyy_dX = torch.autograd.grad(inputs=self.id, outputs=y_id[:,3], grad_outputs=torch.ones_like(y_id[:,3]), retain_graph=True, create_graph=True)[0]
        dsyy_dy = dsyy_dX[:,1]
        dsxy_dX = torch.autograd.grad(inputs=self.id, outputs=y_id[:,4], grad_outputs=torch.ones_like(y_id[:,4]), retain_graph=True, create_graph=True)[0]
        dsxy_dx = dsxy_dX[:, 0] 
        dsxy_dy = dsxy_dX[:, 1]       

        loss_u =  self.rho*(dpsi_dy*dpsi_dyx+dpsi_dx*dpsi_dyy)-(dsxx_dx+dsxy_dy)
        loss_v =  self.rho*(dpsi_dy*dpsi_dxx+dpsi_dx*dpsi_dxy)-(dsxy_dx+dsyy_dy)
        loss_m =  -1*y_id[:,1]+2*self.mu*(dpsi_dyx+dpsi_dxy)-(y_id[:,2]+y_id[:,3])/2+self.mu*(dpsi_dyy+dpsi_dxx)-y_id[:,4]      


        loss_ns = self.criterion(loss_u+loss_v+loss_m, self.pde_sol)
        b=1.8
        loss = b*loss_b+loss_ns 
        loss.backward()
        if self.iter % 1000 == 0: 
            print(self.iter, loss.item())       
            
        self.iter = self.iter + 1        
        return loss
    
    def train(self):
        for i in range(20000):
            self.adam.step(self.loss_func)
        self.optimizer.step(self.loss_func)
   

       

In [48]:
net = PINN()
net.train()

torch.Size([100000, 2])
1000 0.1442553997039795
tensor([-2.3496e-03, -2.3227e-03, -2.2958e-03, -2.2689e-03, -2.2421e-03,
        -2.2152e-03, -2.1884e-03, -2.1616e-03, -2.1348e-03, -2.1080e-03,
        -2.0812e-03, -2.0545e-03, -2.0278e-03, -2.0011e-03, -1.9745e-03,
        -1.9478e-03, -1.9212e-03, -1.8946e-03, -1.8679e-03, -1.8414e-03,
        -1.8148e-03, -1.7882e-03, -1.7617e-03, -1.7352e-03, -1.7087e-03,
        -1.6822e-03, -1.6558e-03, -1.6294e-03, -1.6030e-03, -1.5765e-03,
        -1.5501e-03, -1.5238e-03, -1.4974e-03, -1.4711e-03, -1.4448e-03,
        -1.4185e-03, -1.3922e-03, -1.3658e-03, -1.3396e-03, -1.3133e-03,
        -1.2871e-03, -1.2609e-03, -1.2347e-03, -1.2085e-03, -1.1823e-03,
        -1.1561e-03, -1.1299e-03, -1.1038e-03, -1.0776e-03, -1.0515e-03,
        -1.0253e-03, -9.9919e-04, -9.7304e-04, -9.4696e-04, -9.2085e-04,
        -8.9473e-04, -8.6860e-04, -8.4252e-04, -8.1643e-04, -7.9032e-04,
        -7.6422e-04, -7.3812e-04, -7.1207e-04, -6.8593e-04, -6.5983e-04,
   

In [35]:
#Saving the trained model
torch.save(net, './channel_lhs_8l_0.1m_0.001_0.0001.t7')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Post processing for results analysis

x = torch.arange(0, 5 + 0.01 , 0.01)
y = torch.arange(0, 1 +0.005, 0.005)
outlet= torch.stack(torch.meshgrid(x, y)).reshape(2, -1).T
outlet = outlet.to(net.outlet.device)
outlet.requires_grad = True

model = net.model
model.eval()

y_pred = model(outlet)
dpsi_dX = torch.autograd.grad(inputs=outlet, outputs=y_pred[:,0], grad_outputs=torch.ones_like(y_pred[:,0]), retain_graph=True, create_graph=True)[0]
dpsi_dX[:, 0] = -1*dpsi_dX[:, 0]

fld=torch.cat([outlet, dpsi_dX ,y_pred], 1)
fld=fld.detach().cpu().numpy()

np.savetxt('output.dat', fld)